# Peer-graded Assignment: Capstone Project 
## The Battle of Neighborhoods

# 1) Introduction/Business Problem

The basis of this study is to help a small group of investors planning to open a brewery / restaurant in Toronto. They are interested in building in an area that meets the following criteria:

* A neighborhood with an average to above average total population
* Above average populations of 25-40 year old male and female professionals
* A high concentration of the population having secondary education
* Average to above average median net household incomes

The information gained will assist in chosing the right location by providing data about the population of each neighborhood, in addition to other established venues present in these areas. 

# 2) Data
The necessary information needed by the investing group will come from the following sources:

* __[City of Toronto Neighborhood Profiles](https://www.toronto.ca/city-government/data-research-maps/neighbourhoods-communities/neighbourhood-profiles/)__ for providing an overview of the neighborhoods in Toronto


* __[City of Toronto Open Data Catalogue](https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a)__ : *The Census of Population is held across Canada every five years (the last being in 2016), and collects data about age and sex, families and households, language, immigration and internal migration, ethnocultural diversity, Aboriginal peoples, housing, education, income, and labour.  City of Toronto Neighborhood Profiles use this Census data to provide a portrait of the demographic, social and economic characteristics of the people and households in each City of Toronto neighborhood. The profiles present selected highlights from the data, but these accompanying data files provide the full data set assembled for each neighborhood.*
    
    *In these profiles of the City of Toronto's 140 social planning neighbourhoods. These social planning neighbourhoods were developed by the City of Toronto to help government and community organizations with local planning by providing socio-economic data at a meaningful geographic area. The boundaries of these social planning neighbourhoods are consistent over time, allowing for comparison between Census years. Neighbourhood level data from a variety of other sources are also available through the City's Wellbeing Toronto mapping application and here on the Open Data portal.*
   
   *Each data point in this file is presented for the City's 140 neighbourhoods, as well as for the City of Toronto as a whole. The data is sourced from a number of Census tables released by Statistics Canada. The general Census Profile is the main source table for this data, but other Census tables have also been used to provide additional information.*  __[CSV File](https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv)__ 
   
   
* __[City of Toronto Neighborhood Shapes](https://portal0.cf.opendata.inter.sandbox-toronto.ca/dataset/neighbourhoods/)__ for mapping : __[GeoJSON File](https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=geojson&projection=4326)__


* __[Foursquare API](https://developer.foursquare.com/)__ to collect information on other venues/competitors in the neighborhoods


To assess the neighborhoods and provide guidance to the investors, we will be utilizing the data from the 2016 Toronto Census, Toronto Neighborhood shapes to map the neighborhoods, and the Foursquare API to collect information on venues/competitors in the neighborhoods.

### Import and install the necessary libaries and tools

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported and loaded.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Doug\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.6.14               |           py37_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2019.1.23-0 --> conda-forge::ca-certificates-2019.3.9-hecc5488_0
  openssl              pkgs/main::openssl-1.1.1b-he774522_1 --> conda-forge::openssl-1.1.1b-hfa6e2cd_2

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                         pkgs/main --> conda-forge
  conda      

### Pull in the Toronto Census data file and create a dataframe

In [2]:
# City of Toronto Open Data Catalogue - City of Toronto Neighborhoods Profile Census CSV File

path = 'https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(path,encoding='latin1')
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

### After reviewing the data, create a list of neighbordhoods in Toronto

In [3]:
neighborhoods = list(df.columns.values)
neighborhoods = neighborhoods[5:]
print(neighborhoods)

['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex', 'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor', 'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown', 'Beechborough-Greenbrook', 'Bendale', 'Birchcliffe-Cliffside', 'Black Creek', 'Blake-Jones', 'Briar Hill-Belgravia', 'Bridle Path-Sunnybrook-York Mills', 'Broadview North', 'Brookhaven-Amesbury', 'Cabbagetown-South St. James Town', 'Caledonia-Fairbank', 'Casa Loma', 'Centennial Scarborough', 'Church-Yonge Corridor', 'Clairlea-Birchmount', 'Clanton Park', 'Cliffcrest', 'Corso Italia-Davenport', 'Danforth', 'Danforth East York', 'Don Valley Village', 'Dorset Park', 'Dovercourt-Wallace Emerson-Junction', 'Downsview-Roding-CFB', 'Dufferin Grove', 'East End-Danforth', 'Edenbridge-Humber Valley', 'Eglinton East', 'Elms-Old Rexdale', 'Englemount-Lawrence', 'Eringate-Centennial-West Deane', 'Etobicoke West Mall', 'Flemingdon Park', 'Forest Hill North', 'Forest Hill South', 'Glenfield-Jane Heig

### Create dataframe idexing the neighborhoods of Toronto and populate the dataframe with necessary data

In [4]:
toronto_hoods = pd.DataFrame(index=neighborhoods, columns=["population", "male", "female",'higher_education', "after_tax_income"])

# population = Population 2016 per Census Profile 98-316-X2016001
# m2 = Male: 25 to 29 years
# m3 = Male: 30 to 34 years
# m4 = Male: 35 to 39 years
# f2 = Female: 25 to 29 years
# f3 = Female: 30 to 34 years
# f4 = Female: 35 to 39 years
# higher_education = Total - University certificate, diploma or degree at bachelor level or above for the population aged 25 to 64 years in private households - 25% sample data
# after_tax_income =   After-tax income: Average amount ($)

for index, row in toronto_hoods.iterrows():
    toronto_hoods.at[index, 'population'] = df[index][2]
    toronto_hoods.at[index, 'm2'] = df[index][20]
    toronto_hoods.at[index, 'm3'] = df[index][21]
    toronto_hoods.at[index, 'm4'] = df[index][22]
    toronto_hoods.at[index, 'f2'] = df[index][41]
    toronto_hoods.at[index, 'f3'] = df[index][42]
    toronto_hoods.at[index, 'f4'] = df[index][43]
    toronto_hoods.at[index, 'higher_education'] = df[index][1723]
    toronto_hoods.at[index, 'after_tax_income'] = df[index][2354]
toronto_hoods.reset_index(inplace=True)
toronto_hoods.head()

,index,population,male,female,higher_education,after_tax_income,m2,m3,m4,f2,f3,f4
0,Agincourt North,"29,113",NaN,NaN,4240,"26,955",1015,835,680,1005,935,775
1,Agincourt South-Malvern West,"23,757",NaN,NaN,4615,"27,928",1045,820,625,975,835,715
2,Alderwood,"12,054",NaN,NaN,1980,"39,159",355,410,455,350,430,450
3,Annex,"30,526",NaN,NaN,12640,"80,138",2080,1610,1055,2265,1675,1040
4,Banbury-Don Mills,"27,695",NaN,NaN,8060,"51,874",645,735,735,745,860,895


In [5]:
toronto_hoods.columns = toronto_hoods.columns.str.replace('index', 'neighborhood')
toronto_hoods['population'] = toronto_hoods['population'].str.replace(',','').astype(int)
toronto_hoods['male'] = (toronto_hoods['m2'].astype(int) + toronto_hoods['m3'].astype(int) + toronto_hoods['m4'].astype(int))
toronto_hoods['female'] = (toronto_hoods['f2'].astype(int) + toronto_hoods['f3'].astype(int) + toronto_hoods['f4'].astype(int))
toronto_hoods['higher_education'] = toronto_hoods['higher_education'].astype(int)
toronto_hoods['after_tax_income'] = toronto_hoods['after_tax_income'].str.replace(',','').astype(int)
toronto_hoods.head()

,neighborhood,population,male,female,higher_education,after_tax_income,m2,m3,m4,f2,f3,f4
0,Agincourt North,29113,2530,2715,4240,26955,1015,835,680,1005,935,775
1,Agincourt South-Malvern West,23757,2490,2525,4615,27928,1045,820,625,975,835,715
2,Alderwood,12054,1220,1230,1980,39159,355,410,455,350,430,450
3,Annex,30526,4745,4980,12640,80138,2080,1610,1055,2265,1675,1040
4,Banbury-Don Mills,27695,2115,2500,8060,51874,645,735,735,745,860,895


### Combine male age group demoographics and female age group demographics into male and female categories

In [6]:
toronto_hoods = toronto_hoods.drop(['m2', 'm3', 'm4', 'f2', 'f3', 'f4'], axis=1)
toronto_hoods.head()

,neighborhood,population,male,female,higher_education,after_tax_income
0,Agincourt North,29113,2530,2715,4240,26955
1,Agincourt South-Malvern West,23757,2490,2525,4615,27928
2,Alderwood,12054,1220,1230,1980,39159
3,Annex,30526,4745,4980,12640,80138
4,Banbury-Don Mills,27695,2115,2500,8060,51874


### Pull in the Toronto shape data file, create a dataframe, sort by neighborhood name

In [7]:
path2 = 'https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=csv&projection=4326'
df_2 = pd.read_csv(path2,encoding='latin1')
df_2.sort_values('AREA_NAME').head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
74,1335,25886428,25926736,49885,129,129,Agincourt North (129),Agincourt North (129),NaN,NaN,-79.266712,43.805441,16492689,1.395145e+07,17159.740667,"POLYGON ((-79.24212745783549 43.8024720055488,..."
75,1336,25886449,25926737,49885,128,128,Agincourt South-Malvern West (128),Agincourt South-Malvern West (128),NaN,NaN,-79.265612,43.788658,16492705,1.511736e+07,21320.849547,POLYGON ((-79.25497985384374 43.78122253713403...
76,1337,25886794,25926738,49885,20,20,Alderwood (20),Alderwood (20),NaN,NaN,-79.541611,43.604937,16492721,9.502180e+06,12667.013917,POLYGON ((-79.54865745246562 43.59021610127974...
77,1338,25886874,25926739,49885,95,95,Annex (95),Annex (95),NaN,NaN,-79.404001,43.671585,16492737,5.337192e+06,10513.883143,"POLYGON ((-79.39414141343556 43.6687202679382,..."
78,1339,25886643,25926740,49885,42,42,Banbury-Don Mills (42),Banbury-Don Mills (42),NaN,NaN,-79.349718,43.737657,16492753,1.924897e+07,25141.572290,POLYGON ((-79.33055472634429 43.73978987881326...


### Remove unnessary data and sort our dataframe

In [8]:
df_3 = df_2.filter(['AREA_NAME','LONGITUDE','LATITUDE'], axis=1)
df_3.columns = map(str.lower, df_3.columns)
df_3.columns = df_3.columns.str.replace('area_name', 'neighborhood')
df_3 = df_3.sort_values(by=['neighborhood'])
df_3 = df_3.reset_index()
df_3 = df_3.drop(['index'], axis=1)
df_3.head()

,neighborhood,longitude,latitude
0,Agincourt North (129),-79.266712,43.805441
1,Agincourt South-Malvern West (128),-79.265612,43.788658
2,Alderwood (20),-79.541611,43.604937
3,Annex (95),-79.404001,43.671585
4,Banbury-Don Mills (42),-79.349718,43.737657


### Merge the two dataframes together to get a working dataframe

In [9]:
df_pos = pd.merge(df_3, toronto_hoods, left_index=True, right_index=True, how='inner')
df_pos = df_pos.drop(['neighborhood_y'], axis=1)
df_pos.columns = df_pos.columns.str.replace('neighborhood_x', 'neighborhood')
df_pos.head()

,neighborhood,longitude,latitude,population,male,female,higher_education,after_tax_income
0,Agincourt North (129),-79.266712,43.805441,29113,2530,2715,4240,26955
1,Agincourt South-Malvern West (128),-79.265612,43.788658,23757,2490,2525,4615,27928
2,Alderwood (20),-79.541611,43.604937,12054,1220,1230,1980,39159
3,Annex (95),-79.404001,43.671585,30526,4745,4980,12640,80138
4,Banbury-Don Mills (42),-79.349718,43.737657,27695,2115,2500,8060,51874


### Calculate medians for Toronto neighborhoods to assist in creating the scoring system

In [10]:
pop_med = df_pos['population'].median()
male_med = df_pos['male'].median()
female_med = df_pos['female'].median()
edu_med = df_pos['higher_education'].median()
income_med = df_pos['after_tax_income'].median()

print('Median Population:',pop_med)
print('Median Male: ',male_med)
print('Median Female: ',female_med)
print('Median Higher Education: ',edu_med)
print('Median After Tax Income: ',income_med)

Median Population: 16749.5
Median Male:  1800.0
Median Female:  1952.5
Median Higher Education:  4122.5
Median After Tax Income:  36538.5


### Calculate Neighboorhood Scores based off Toronto neighboorhood medians
* Scores are calulated based on the importance of each category established by the investor group

In [11]:
df_score = pd.DataFrame(columns=["neighborhood","pop_score", "male_score", "female_score", "edu_score", "income_score", "total_score"])
df_score['neighborhood'] = df_pos['neighborhood']

# Each score category was derived by taking the value of the category, dividing by the median of said category, 
# and finally multiplying value by the importance factor given by the investors. 

pop_score = [] # 15% importance
for x in df_pos['population']:
  if x / pop_med >0:
    pop_score.append((x / pop_med)*.15)
  else:
    pop_score.append(0)
df_score['pop_score'] = pop_score

male_score = [] # 25% importance
for z in df_pos['male']:
  if z / male_med >0:
    male_score.append((z / male_med)*.25)
  else:
    male_score.append(0)
df_score['male_score'] = male_score

female_score = [] # 25% importance
for z in df_pos['female']:
  if z / female_med >0:
    female_score.append((z / female_med)*.25)
  else:
    female_score.append(0)
df_score['female_score'] = female_score

edu_score = [] # 15% importance
for z in df_pos['higher_education']:
  if z / edu_med >0:
    edu_score.append((z / edu_med)*.15)
  else:
    edu_score.append(0)
df_score['edu_score'] = edu_score

income_score = [] # 20% importance
for z in df_pos['after_tax_income']:
  if z / income_med >0:
    income_score.append((z / income_med)*.2)
  else:
    income_score.append(0)
df_score['income_score'] = income_score

# Add each category to get overall neighborhood score
df_score['total_score'] = round(df_score.iloc[:,-7:].sum(axis=1),2)

df_score.head()

,neighborhood,pop_score,male_score,female_score,edu_score,income_score,total_score
0,Agincourt North (129),0.260721,0.351389,0.347631,0.154275,0.147543,1.26
1,Agincourt South-Malvern West (128),0.212756,0.345833,0.323303,0.167920,0.152869,1.20
2,Alderwood (20),0.107949,0.169444,0.157490,0.072044,0.214344,0.72
3,Annex (95),0.273375,0.659028,0.637644,0.459915,0.438650,2.47
4,Banbury-Don Mills (42),0.248022,0.293750,0.320102,0.293269,0.283942,1.44


### Merge Into full Toronto neighborhoods working dataframe and drop unnecessary data

In [12]:
df_final = pd.merge(df_pos, df_score, left_index=True, right_index=True, how='inner')
df_final = df_final.drop(['neighborhood_y'], axis=1)
df_final.columns = df_final.columns.str.replace('neighborhood_x', 'neighborhood')
df_final = df_final.drop(columns = ['population','male','female','higher_education','after_tax_income'])
df_final.head()

,neighborhood,longitude,latitude,pop_score,male_score,female_score,edu_score,income_score,total_score
0,Agincourt North (129),-79.266712,43.805441,0.260721,0.351389,0.347631,0.154275,0.147543,1.26
1,Agincourt South-Malvern West (128),-79.265612,43.788658,0.212756,0.345833,0.323303,0.167920,0.152869,1.20
2,Alderwood (20),-79.541611,43.604937,0.107949,0.169444,0.157490,0.072044,0.214344,0.72
3,Annex (95),-79.404001,43.671585,0.273375,0.659028,0.637644,0.459915,0.438650,2.47
4,Banbury-Don Mills (42),-79.349718,43.737657,0.248022,0.293750,0.320102,0.293269,0.283942,1.44


### Creating a map to evaluate neighborhood scores

In [13]:
!wget --quiet https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=geojson&projection=4326 
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.
'projection' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
toronto_geo = r'Neighbourhoods.geojson'

address = 'Toronto'

geolocator = Nominatim(user_agent="torcan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5, tiles = 'Mapbox Bright')

In [15]:
# create a numpy array of length 5 and has linear spacing from the minium total score to the maximum total score
threshold_scale = np.linspace(df_final['total_score'].min(),
                              df_final['total_score'].max(),
                              5, dtype=float)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum score

# let Folium determine the scale.
map_toronto.choropleth(
    geo_data=toronto_geo,
    data=df_final,
    columns=['neighborhood', 'total_score'],
    key_on='feature.properties.AREA_NAME',
    threshold_scale=threshold_scale,
    fill_color='Blues', 
    fill_opacity=0.8, 
    line_opacity=0.5,
    legend_name='Toronto Neighborhood Scores',
    reset=True
)

for lat, lng, neighborhood, total_score in zip(df_final['latitude'], df_final['longitude'], df_final['neighborhood'], df_final['total_score']):
    label = '{}, Total Score: {}'.format(neighborhood, total_score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='#071521',
        fill=False,
        fill_color='#071521',
        fill_opacity=0.1,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
top5_hoods = df_final.sort_values(by=['total_score'],ascending=False).head(5)
top5_hoods = top5_hoods.reset_index()
top5_hoods = top5_hoods.drop(['index'], axis=1)
top5_hoods

,neighborhood,longitude,latitude,pop_score,male_score,female_score,edu_score,income_score,total_score
0,Waterfront Communities-The Island (77),-79.377202,43.633880,0.590283,2.525000,2.151729,1.298059,0.297177,6.86
1,Niagara (82),-79.412420,43.636681,0.279232,1.230556,1.142125,0.573257,0.299974,3.53
2,Willowdale East (51),-79.401484,43.770602,0.451661,1.027083,1.013444,0.771195,0.201103,3.46
3,Church-Yonge Corridor (75),-79.379017,43.659649,0.280665,0.961806,0.748399,0.500849,0.235582,2.73
4,Islington-City Centre West (14),-79.543317,43.633463,0.393728,0.796528,0.748399,0.455549,0.232905,2.63


### Define Foursquare credentials and version

In [17]:
CLIENT_ID = 'K4MGJAKB2V2LA2AGWVKNR5VZFK5D5XWBDIYPCB34JUUX3WVR' # your Foursquare ID
CLIENT_SECRET = 'VQDBORZ1YD0NI5D2T53JWQXS205REEFYU1NWV1NT42KU0R3D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4MGJAKB2V2LA2AGWVKNR5VZFK5D5XWBDIYPCB34JUUX3WVR
CLIENT_SECRET:VQDBORZ1YD0NI5D2T53JWQXS205REEFYU1NWV1NT42KU0R3D


In [18]:
neighborhood_latitude = top5_hoods.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = top5_hoods.loc[0, 'longitude'] # neighborhood longitude value
neighborhood_name = top5_hoods.loc[0, 'neighborhood'] # neighborhood name

### Now, let's get the top venues that are in the Top 5 Neighborhoods within a radius of 2000 meters

In [19]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius=2000): # 2000 meters = 1.25 miles
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood_latitude', 
                  'neighborhood_longitude', 
                  'venue', 
                  'venue_latitude', 
                  'venue_longitude', 
                  'venue_category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe called *top5_hoods_venues*.

In [20]:
top5_hoods_venues = getNearbyVenues(names=top5_hoods['neighborhood'],
                                   latitudes=top5_hoods['latitude'],
                                   longitudes=top5_hoods['longitude']
                                  )

Waterfront Communities-The Island (77)
Niagara (82)
Willowdale East (51)
Church-Yonge Corridor (75)
Islington-City Centre West (14)


### Establish how many venues were returned for each neighborhood

In [21]:
top5_hoods_venues.groupby('neighborhood').count()

,neighborhood_latitude,neighborhood_longitude,venue,venue_latitude,venue_longitude,venue_category
neighborhood,,,,,,
Church-Yonge Corridor (75),100,100,100,100,100,100
Islington-City Centre West (14),45,45,45,45,45,45
Niagara (82),100,100,100,100,100,100
Waterfront Communities-The Island (77),100,100,100,100,100,100
Willowdale East (51),38,38,38,38,38,38


### Now we will analyze each neighborhood

In [22]:
# one hot encoding
top5_hoods_onehot = pd.get_dummies(top5_hoods_venues[['venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
top5_hoods_onehot['neighborhood'] = top5_hoods_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [top5_hoods_onehot.columns[-1]] + list(top5_hoods_onehot.columns[:-1])
top5_hoods_onehot = top5_hoods_onehot[fixed_columns]

print(top5_hoods_onehot.shape)
top5_hoods_onehot.head()

(383, 137)


,neighborhood,Airport,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Liquor Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Soccer Stadium,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Track,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Waterfront Communities-The Island (77),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Waterfront Communities-The Island (77),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Waterfront Communities-The Island (77),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Waterfront Communities-The Island (77),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Waterfront Communities-The Island (77),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Group the rows by neighborhood by taking the sum of each category
* Drop duplicate neighborhood category

In [23]:
top5_hoods_onehot.drop(['Neighborhood'], axis=1, inplace=True)
top5_hoods_grouped = top5_hoods_onehot.groupby('neighborhood').sum().reset_index()
top5_hoods_grouped.columns = map(str.lower, top5_hoods_grouped.columns)
top5_hoods_grouped

,neighborhood,airport,american restaurant,aquarium,art gallery,art museum,arts & crafts store,asian restaurant,athletics & sports,bbq joint,bakery,bar,baseball stadium,basketball stadium,beach,beer bar,bistro,bookstore,breakfast spot,brewery,bubble tea shop,burger joint,burrito place,butcher,café,caribbean restaurant,chinese restaurant,climbing gym,clothing store,cocktail bar,coffee shop,comedy club,comfort food restaurant,comic shop,concert hall,convenience store,cosmetics shop,creperie,cuban restaurant,dance studio,deli / bodega,dessert shop,diner,dog run,donut shop,electronics store,event space,farmers market,fast food restaurant,filipino restaurant,food truck,french restaurant,fried chicken joint,furniture / home store,garden,gastropub,gift shop,gourmet shop,greek restaurant,grocery store,gym,gym / fitness center,harbor / marina,historic site,history museum,hotel,hotel bar,ice cream shop,indian restaurant,indonesian restaurant,intersection,italian restaurant,japanese restaurant,juice bar,korean restaurant,lake,liquor store,mediterranean restaurant,men's store,mexican restaurant,middle eastern restaurant,miscellaneous shop,monument / landmark,movie theater,museum,music venue,nail salon,new american restaurant,office,opera house,optical shop,organic grocery,paper / office supplies store,park,performing arts venue,pet store,pizza place,playground,plaza,poke place,portuguese restaurant,pub,racetrack,ramen restaurant,restaurant,salad place,salon / barbershop,sandwich place,scenic lookout,seafood restaurant,shopping mall,skating rink,smoke shop,soccer stadium,spa,speakeasy,sporting goods shop,sports bar,steakhouse,supermarket,sushi restaurant,taco place,tapas restaurant,tea room,thai restaurant,theater,theme restaurant,track,train station,turkish restaurant,university,vegetarian / vegan restaurant,vietnamese restaurant,wine bar,women's store,yoga studio
0,Church-Yonge Corridor (75),0,2,0,1,1,1,0,0,1,0,0,0,0,0,0,0,1,2,0,1,0,1,0,5,0,0,0,1,0,7,0,0,1,1,0,3,2,0,2,0,0,2,0,0,0,0,2,0,0,1,0,0,0,1,5,0,0,0,0,2,1,0,1,0,2,0,0,1,0,0,4,5,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,2,0,2,0,0,2,0,2,3,0,1,1,0,1,1,0,1,0,0,1,0,0,2,1,0,0,1,1,3,2,1,0,0,0,1,2,0,0,0,1
1,Islington-City Centre West (14),0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,0,2,0,0,4,0,0,1,0,0,1,2,0,2,0,0,0,0,3,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,1,0,1,1
2,Niagara (82),0,2,0,1,0,1,2,1,0,5,3,0,0,0,2,0,1,0,1,0,1,0,1,8,1,0,0,0,1,4,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,3,0,0,2,0,1,0,2,1,1,1,0,2,1,1,0,0,0,5,0,1,1,0,0,0,1,1,0,1,0,1,0,2,1,2,0,0,1,0,1,4,0,0,3,0,0,1,0,0,1,1,2,0,1,1,0,1,0,0,1,2,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2,0,1,3
3,Waterfront Communities-The Island (77),1,0,2,1,0,0,0,0,0,1,1,2,1,1,1,2,0,0,2,1,0,0,0,6,0,1,0,0,1,6,1,0,0,2,0,0,0,0,1,2,0,0,0,0,0,1,2,0,0,1,1,1,0,1,1,0,0,1,0,3,0,0,0,1,5,0,1,0,0,0,4,2,0,0,1,1,1,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,4,1,0,1,0,1,0,0,1,0,0,1,1,0,0,3,0,0,1,0,0,1,1,1,1,2,1,0,0,0,0,2,2,0,1,1,0,0,1,0,1,0,1
4,Willowdale East (51),0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,1,2,0,2,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,1,0,1,0,4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,2,2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0


### Drop venues realted to retail and personal care categories, and focus on activity centered venues

In [24]:
top5_hoods_grouped.drop(['arts & crafts store','athletics & sports', 'bookstore', 'clothing store','comic shop','convenience store', 'cosmetics shop', 'deli / bodega', 'electronics store', 'furniture / home store','grocery store', 'intersection', 'liquor store', "men's store", 'miscellaneous shop', 'nail salon', 'optical shop', 'organic grocery', 'paper / office supplies store', 'salon / barbershop', 'spa', 'sporting goods shop','supermarket', "women's store"], axis=1, inplace=True)
top5_hoods_grouped.shape

(5, 112)

In [25]:
top5_hoods_grouped['total venues'] = top5_hoods_grouped.iloc[:,-111:].sum(axis=1)
top5_hoods_grouped

,neighborhood,airport,american restaurant,aquarium,art gallery,art museum,asian restaurant,bbq joint,bakery,bar,baseball stadium,basketball stadium,beach,beer bar,bistro,breakfast spot,brewery,bubble tea shop,burger joint,burrito place,butcher,café,caribbean restaurant,chinese restaurant,climbing gym,cocktail bar,coffee shop,comedy club,comfort food restaurant,concert hall,creperie,cuban restaurant,dance studio,dessert shop,diner,dog run,donut shop,event space,farmers market,fast food restaurant,filipino restaurant,food truck,french restaurant,fried chicken joint,garden,gastropub,gift shop,gourmet shop,greek restaurant,gym,gym / fitness center,harbor / marina,historic site,history museum,hotel,hotel bar,ice cream shop,indian restaurant,indonesian restaurant,italian restaurant,japanese restaurant,juice bar,korean restaurant,lake,mediterranean restaurant,mexican restaurant,middle eastern restaurant,monument / landmark,movie theater,museum,music venue,new american restaurant,office,opera house,park,performing arts venue,pet store,pizza place,playground,plaza,poke place,portuguese restaurant,pub,racetrack,ramen restaurant,restaurant,salad place,sandwich place,scenic lookout,seafood restaurant,shopping mall,skating rink,smoke shop,soccer stadium,speakeasy,sports bar,steakhouse,sushi restaurant,taco place,tapas restaurant,tea room,thai restaurant,theater,theme restaurant,track,train station,turkish restaurant,university,vegetarian / vegan restaurant,vietnamese restaurant,wine bar,yoga studio,total venues
0,Church-Yonge Corridor (75),0,2,0,1,1,0,1,0,0,0,0,0,0,0,2,0,1,0,1,0,5,0,0,0,0,7,0,0,1,2,0,2,0,2,0,0,0,2,0,0,1,0,0,1,5,0,0,0,2,1,0,1,0,2,0,0,1,0,4,5,1,0,0,0,0,1,0,0,0,0,1,0,1,2,0,0,2,0,2,0,0,2,0,2,3,0,1,0,1,1,0,1,0,1,0,2,0,0,1,1,3,2,1,0,0,0,1,2,0,0,1,89
1,Islington-City Centre West (14),0,1,0,0,0,0,0,3,0,0,0,0,0,0,2,0,0,4,0,0,1,0,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,1,0,1,28
2,Niagara (82),0,2,0,1,0,2,0,5,3,0,0,0,2,0,0,1,0,1,0,1,8,1,0,0,1,4,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,2,0,1,2,1,1,1,0,2,1,1,0,0,5,0,1,1,0,0,1,0,0,1,0,2,2,0,0,4,0,0,3,0,0,1,0,0,1,1,2,0,1,0,1,0,0,1,2,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2,0,3,86
3,Waterfront Communities-The Island (77),1,0,2,1,0,0,0,1,1,2,1,1,1,2,0,2,1,0,0,0,6,0,1,0,1,6,1,0,2,0,0,1,0,0,0,0,1,2,0,0,1,1,1,1,1,0,0,1,3,0,0,0,1,5,0,1,0,0,4,2,0,0,1,1,0,0,1,1,1,0,1,1,0,4,1,0,1,0,1,0,0,1,0,0,1,1,0,3,0,0,1,0,0,1,1,2,0,0,0,0,2,2,0,1,1,0,0,1,0,1,1,93
4,Willowdale East (51),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,1,2,0,2,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,31
